In [1]:
import numpy as np
from tensorflow import keras
from tools import signature as sig
from sklearn.preprocessing import StandardScaler

from ipywidgets import interact, fixed
import ipywidgets as widgets
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from ipywidgets import Image, Layout
import PIL.Image
import io

<h1> Titre </h1>

In [2]:
#@interact(keep=fixed([1,3,7]), n=widgets.IntSlider(min=50, max=300, step=10, value=100))
keep = [1, 7]
n = 50

keep.sort()
(x_train_origin, y_train_origin), (x_test_origin, y_test_origin)= \
    keras.datasets.mnist.load_data(path='all_mnist.npz')

(x, y), (x_test, y_test) = sig.pretraitement(x_train_origin, y_train_origin, 
                                             x_test_origin, y_test_origin, 
                                             keep=keep, n=n, verbose=False)

layers = [32, 16]
model = sig.trained_model(x, y, hidden_layers=layers, verbose=False)

loss, acc = model.evaluate(x_test, y_test)
if False:
    print("loss =", loss, ", accuracy =",acc)

predictions = sig.get_preditions(model, x, keep, verbose=False)
labels = [keep[np.argmax(l)] for l in y]

hidden_layers = model.get_hidden_layers_outputs(x)

    # Standardization 
    # A noter que sans cette standardisation on a de plus beau score silhouette
for i in range(len(hidden_layers)):
    hidden_layers[i] = StandardScaler().fit_transform(hidden_layers[i])

best_k = sig.get_best_k(hidden_layers, kmax = len(keep),verbose=False)

clusters = sig.clustering(hidden_layers, best_k)


# Build colorscale
colorscale = {
    'color': np.zeros(len(x), dtype='uint8'),
    'colorscale': ['grey', '#ee1717', '#7201a8', '#17a60d', '#291ae0', '#f418ff', '#e3ae04', 
                   '#fb5e09', 'black', '#0ed6e1', '#d35400'],
    'cmin': 0,
    'cmax': 10
}


traces = sig.umap_plot(x, labels, predictions, hidden_layers, clusters, colorscale)

#sankey = sig.sankey(labels, keep, best_k, clusters, predictions)
parcats = sig.parcats(labels, keep, best_k, clusters, predictions, colorscale)

traces.append(parcats)

INFO:tensorflow:Assets written to: ./models/trained_model_2_16/assets


In [9]:
fig = make_subplots(rows=3, figure=go.FigureWidget(), specs=[[{'type': 'xy'}],
                                                             [{'type': 'xy'}],
                                                            [{'type': 'parcats'}]])

fig.add_traces(traces, rows=[1, 2, 3], cols=[1, 1, 1])

fig.update_layout(height=800,
                  dragmode='lasso', hovermode='closest')

def image_to_byte_array(image):
    imgByteArr = io.BytesIO()
    image.save(imgByteArr, format='png')
    imgByteArr = imgByteArr.getvalue()
    return imgByteArr

def to_image(x):
    return  image_to_byte_array(
                PIL.Image.fromarray(
                    (x.reshape((len(x_train_origin[0]) , len(x_train_origin[0][0]))) * 255).astype(np.uint8)
                ))

image_widget = Image(
    value=to_image(x[0]),
    layout=Layout(height='100px', width='100px')
)

# Build color selection widget
color_toggle = widgets.ToggleButtons(
    options=['None', 'Red', 'Purple', 'Green', 'Blue', 'Pink', 
             'Yellow', 'Orange','Black' ,'Aqua', 'Brown'],
    index=1, description='Color:', disabled=False)

# Update color callback
def update_color(trace, points, state):
    # Compute new color array
    new_color = np.array(trace.marker.color)
    new_color[points.point_inds] = color_toggle.index
    with fig.batch_update():
        # Update scatter color
        for i in range(len(fig.data)-1):
            fig.data[i].marker.color = new_color

        # Update parcats colors
        fig.data[len(fig.data)-1].line.color = new_color
        
def print_image(trace, points, state):     
    # Image update
    if (points.point_inds != []):
        image_widget.value = to_image(x[points.point_inds[0]])


# Register callback on scatter selection...
fig.data[0].on_selection(update_color)
fig.data[1].on_selection(update_color)

fig.data[0].on_hover(print_image)
fig.data[1].on_hover(print_image)

# and parcats click
#fig.data[2].on_click(update_color)

# Display figure
widgets.VBox([color_toggle, image_widget, fig])